<a href="https://colab.research.google.com/github/kplr-training/Airflow/blob/main/Ateliers/Solutions/10-Data%20Aware%20Scheduling%20-%20File.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Aware Scheduling : Création et consommation d'un dataset pour le data-aware



![dataaware](https://user-images.githubusercontent.com/123757632/231875385-9cf19dc7-c1fc-4b6a-8076-ed5417389188.png)

1 . Importation des modules nécessaires 

In [ ]:
from datetime import datetime, timedelta
from airflow import DAG, Dataset
from airflow.operators.bash_operator import BashOperator

Les modules datetime, timedelta et Dataset de la bibliothèque Airflow sont importés, ainsi que BashOperator qui permet l'exécution de commandes bash dans les tâches.

2 . Définition des arguments par défaut :

In [ ]:
default_args = {
    'owner': 'airflow',
    'depends_on_past': False    
}

Le dictionnaire default_args est défini pour spécifier les arguments par défaut pour les deux DAGs qui vont suivre. Dans cet exemple, seul l'argument owner est défini pour identifier l'utilisateur responsable des DAGs.

3 . Définition d'un Dataset :

In [ ]:
my_dataset = Dataset('/tmp/data.csv')

Un objet my_dataset est défini pour représenter le fichier de données data.csv. Le chemin du fichier est spécifié en tant qu'argument pour l'objet Dataset.

4 . Définition du premier DAG :

In [ ]:
with DAG(dag_id='producer_aware', 
        start_date = datetime.now(),
        schedule_interval = timedelta(minutes=5),
        default_args=default_args) as dag:

Le premier DAG est défini à l'aide de l'instruction with DAG(). Le DAG est nommé producer_aware, la date et l'heure de début sont spécifiées avec start_date, et l'exécution est planifiée toutes les cinq minutes avec schedule_interval. Les arguments par défaut définis précédemment sont utilisés.

5 . Définition de la première tâche :

In [ ]:
generate_dataset = BashOperator(
        task_id='generate_dataset',
        outlets=[my_dataset],
        bash_command='echo "data1,data2,data3\n" >> /tmp/data.csv',
)

Une tâche generate_dataset est définie à l'aide de BashOperator. La tâche utilise une commande bash pour ajouter une ligne de données "data1,data2,data3" au fichier data.csv. L'argument outlets est utilisé pour spécifier que la tâche produit des données stockées dans l'objet my_dataset. Un identifiant task_id est également attribué à la tâche pour la référencer dans le DAG.

6 . Définition de la deuxième tâche :

In [ ]:
check_file_task = BashOperator(
        task_id='check_file',
        bash_command='cat /tmp/data.csv',
)

Une deuxième tâche check_file_task est définie à l'aide de BashOperator. La tâche utilise une commande bash pour afficher le contenu du fichier data.csv. Un identifiant task_id est également attribué à la tâche pour la référencer dans le DAG.

7 . Liaison des tâches :

In [ ]:
generate_dataset >> check_file_task

L'opérateur de dépendance >> est utilisé pour lier la tâche generate_dataset à la tâche check_file_task. Cela signifie que la tâche generate_dataset doit être exécutée avant la tâche check_file_task.

8 . Définition du deuxième DAG :

In [ ]:
with DAG(dag_id='consumer_aware', 
        schedule = [my_dataset],
        start_date = datetime.now(),
        default_args=default_args) as dag:

Un deuxième DAG appelé consumer_aware est défini pour consommer les données produites par le premier DAG (producer_aware) en utilisant l'objet my_dataset.

9 . Définition d'une troisième tâche :

In [ ]:
consume_dataset = BashOperator(
        task_id="consume_dataset",
        bash_command="cat /tmp/data.csv",
        retries=3,
)

Une troisième tâche appelée consume_dataset est définie en utilisant BashOperator. Cette tâche utilise une commande bash pour afficher le contenu du fichier data.csv. Le nombre maximal de tentatives pour exécuter la tâche en cas d'échec est également spécifié en utilisant l'argument retries.

Execution du Dags : 

- DAG 1 : 

![image](https://user-images.githubusercontent.com/123757632/231904594-9fd8e908-fc7d-49ce-909a-49c89a9d0105.png)

- DAG 2 :    

![image](https://user-images.githubusercontent.com/123757632/231904661-100e17ff-2862-4d12-b9df-5692b0ef9bdf.png)


Dataset : 

![image](https://user-images.githubusercontent.com/123757632/231904765-fac4d227-fbe1-4ff5-87a8-f53c73931d46.png)